<a href="https://colab.research.google.com/github/takzen/pytorch-black-belt/blob/main/notebooks/34_Bottleneck_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# --------------------------------------------------------------
# ☁️ COLAB SETUP (Automatyczna instalacja środowiska)
# --------------------------------------------------------------
import sys
import os

# Sprawdzamy, czy jesteśmy w Google Colab
if 'google.colab' in sys.modules:
    print('☁️ Wykryto środowisko Google Colab. Konfiguruję...')

    # 1. Pobieramy plik requirements.txt bezpośrednio z repozytorium
    !wget -q https://raw.githubusercontent.com/takzen/ai-engineering-handbook/main/requirements.txt -O requirements.txt

    # 2. Instalujemy biblioteki
    print('⏳ Instaluję zależności (to może chwilę potrwać)...')
    !pip install -q -r requirements.txt

    print('✅ Gotowe! Środowisko jest zgodne z repozytorium.')
else:
    print('💻 Wykryto środowisko lokalne. Zakładam, że masz już uv/venv.')


# 🥋 Lekcja 34: Profilowanie (Gdzie ucieka czas?)

Twój model trenuje się wolno. Dlaczego?
Zamiast zgadywać, użyj **`torch.profiler`**.

Profiler śledzi każde wywołanie operacji (Operator) w PyTorch i mierzy dwa czasy:
1.  **CPU Time:** Ile czasu procesor spędził na wydawaniu rozkazu.
2.  **CUDA Time:** Ile czasu karta graficzna faktycznie liczyła.

**Kluczowe pojęcia w tabeli wyników:**
*   **Self Time:** Czas spędzony w *tej konkretnej* funkcji (bez jej "dzieci").
*   **Total Time:** Czas spędzony w funkcji i wszystkich podfunkcjach, które wywołała.

Jeśli `Self CPU` jest wysokie -> Masz wolny kod Pythona (pętle, listy).
Jeśli `Self CUDA` jest wysokie -> Masz ciężką matematykę (duże macierze).

In [1]:
import torch
import torch.nn as nn
from torch.profiler import profile, record_function, ProfilerActivity

# Konfiguracja
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Profilujemy na: {device}")

# Jeśli CPU, to nie będziemy mieli kolumn CUDA, ale logika jest ta sama.
activities = [ProfilerActivity.CPU]
if device == "cuda":
    activities.append(ProfilerActivity.CUDA)

Profilujemy na: cuda


## Symulacja: Model z "Wąskim Gardłem"

Stworzymy sieć, która ma 3 części:
1.  **Szybka:** Małe mnożenie macierzy.
2.  **Wolna (Matematycznie):** Gigantyczne mnożenie macierzy (obciąża GPU).
3.  **Głupia (Pythonowa):** Pętla `for` wewnątrz modelu (obciąża CPU i blokuje GPU).

Zobaczymy, czy Profiler to wykryje.

In [2]:
class SlowModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.fast_layer = nn.Linear(100, 100)
        self.heavy_layer = nn.Linear(4000, 4000) # 16 mln parametrów!

    def forward(self, x):
        # 1. Część Szybka
        # record_function nadaje nazwę temu blokowi w raporcie
        with record_function("1_FAST_PART"):
            x = self.fast_layer(x)
            x = torch.relu(x)
        
        # 2. Część Głupia (Pętla w Pythonie)
        # To zabija wydajność, bo GPU czeka na Pythona
        with record_function("2_STUPID_LOOP"):
            # Symulujemy bezsensowną operację
            for _ in range(100): 
                x = x + 0.001
        
        # 3. Część Ciężka (Duże macierze)
        # Tu GPU powinno się napocić
        with record_function("3_HEAVY_MATH"):
            # Rozdmuchujemy x, żeby pasował do dużej warstwy
            x_big = x.repeat(1, 40) 
            x_out = self.heavy_layer(x_big)
            
        return x_out

model = SlowModel().to(device)
dummy_input = torch.randn(128, 100).to(device)

print("Model gotowy. Czas na rentgen.")

Model gotowy. Czas na rentgen.


## Uruchomienie Profilera

Używamy `with profile(...)`.
*   `record_shapes=True`: Zapisuje wymiary tensorów (pomaga znaleźć, gdzie zjadamy pamięć).
*   `with_stack=True`: Zapisuje, w której linijce kodu to się stało.

**Ważne:** Pierwsze przejście przez sieć jest zawsze wolne (rozgrzewka/alokacja pamięci). Profiler to pokaże.

In [3]:
# Uruchamiamy profilowanie
with profile(activities=activities, record_shapes=True) as prof:
    with record_function("model_inference"):
        model(dummy_input)

print("Profilowanie zakończone. Generowanie raportu...")

Profilowanie zakończone. Generowanie raportu...


## Analiza Wyników

Wyświetlimy tabelę posortowaną według czasu **CUDA Total** (jeśli masz GPU) lub **CPU Total**.

Czego szukamy?
1.  **`2_STUPID_LOOP`**: Powinno mieć wysoki czas CPU, a niski lub zerowy czas CUDA (bo to Python mieli).
2.  **`3_HEAVY_MATH`**: Powinno mieć wysoki czas CUDA (bo to ciężka macierz).
3.  **`1_FAST_PART`**: Powinno być na dole listy.

In [4]:
# Sortujemy po czasie GPU (lub CPU jeśli brak GPU)
sort_key = "cuda_time_total" if device == "cuda" else "cpu_time_total"

print(prof.key_averages().table(sort_by=sort_key, row_limit=15))

# Jeśli tabela jest nieczytelna, spójrz na nazwy, które sami nadaliśmy (1_, 2_, 3_)

--------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls  
--------------------  ------------  ------------  ------------  ------------  ------------  ------------  
     model_inference         0.22%     224.300us       100.00%     101.516ms     101.516ms             1  
         1_FAST_PART         0.41%     411.600us        80.46%      81.679ms      81.679ms             1  
        aten::linear         0.19%     196.800us        58.45%      59.339ms      29.670ms             2  
             aten::t         0.21%     211.800us         0.54%     549.400us     274.700us             2  
     aten::transpose         0.31%     317.600us         0.33%     337.600us     168.800us             2  
    aten::as_strided         0.03%      30.000us         0.03%      30.000us       5.000us             6  
         aten::addmm        57.72%   

## 🥋 Black Belt Summary

Jak czytać ten raport?

1.  **Addmm (Matrix Multiply):** To zazwyczaj `nn.Linear`. Jeśli zajmuje 90% czasu CUDA -> Zmniejsz model lub Batch Size.
2.  **Aten::add / Aten::mul (Drobne operacje):** Jeśli widzisz ich tysiące i zajmują dużo czasu CPU -> Masz pętlę `for` w kodzie. Użyj `torch.compile` (Lekcja 33) lub przepisz to wektorowo.
3.  **Memcpy (Kopiowanie):** Jeśli `to(device)` jest na szczycie -> Użyj `num_workers` i `pin_memory` (Lekcja 18).

**Zasada optymalizacji:**
Zawsze najpierw naprawiaj to, co jest na szczycie listy ("Low Hanging Fruit"). Przyspieszanie małej warstwy (FAST_PART) nie ma sensu, jeśli 90% czasu zjada HEAVY_MATH.